In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

cache_dir = LocalFileStore('./.cache/')

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.7,
    streaming=True,
    verbose=True
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/chapter_one.pdf')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='refine',
    retriever=vectorstore.as_retriever(),
)

In [27]:
chain.run("Describe Winston Smith")

'Winston Smith is a small, frail-looking man with a meager physique, characterized by his blue overalls, which serve as the uniform of the Party. He has very fair hair and a naturally sanguine face, though his skin appears roughened by coarse soap, blunt razor blades, and the cold winter that has just ended. His appearance reflects a sense of fragility and the oppressive environment he lives in.\n\nThroughout the novel, Winston is portrayed as a deeply introspective and increasingly rebellious individual, secretly harboring thoughts of dissent in a society where independent thinking is forbidden. Despite his physical weakness, he possesses a keen mind and a capacity for critical thought, which makes him both vulnerable and courageous in his quiet resistance against the Party’s totalitarian control.\n\nThe novel depicts Winston’s life under constant surveillance and repression. He works at the Ministry of Truth, where he alters historical records to fit the Party’s narrative. He lives i